
Inspired by Quix's [simple-kafka-python](https://github.com/quixio/simple-kafka-python/tree/main).

# TFL Tube Stream

## API docs
https://content.tfl.gov.uk/trackernet-data-services-guide-beta.pdf older one I think  
https://api-portal.tfl.gov.uk/api-details#api=Line&operation=Line_ArrivalsByPathIds&definition=Tfl-41 from portal

In [44]:
import requests
import json
import logging
import time

logging.basicConfig(level="DEBUG")

def get_stop_point_info(stop_point_name,mode="tube"):
    if mode not in ["tube", "bus"]:
        raise ValueError("Invalid mode. Mode must be 'tube' or 'bus'.")
    
    response = requests.get(
        "https://api.tfl.gov.uk/StopPoint/Search/" + stop_point_name,
        params = {
            'app_key': app_key
        }
    )
    response_data = response.json()


    results = []


    matches = response_data['matches'] # 'matches' is part of the response

    for match in matches:
        if mode in match['modes']:
            results.append({
                'name': match['name'],
                'id': match['id'],
            })
    return results


def get_tube_arrivals(line_id, stop_point_id, direction, app_key, retries=3, backoff=5):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
    params = {
            'direction': direction,
            'app_key': app_key,
        }
    data = {}
    for attempt in range(retries):
        try:
            response = requests.get(url, params)
            response.raise_for_status()  # Raise an exception for HTTP errors
            data = response.json()

            logging.debug(f"API response: {data}")
            if isinstance(data, list) and len(data) > 0:
                return data[0]  
            
        except requests.exceptions.RequestException as e:
            logging.warning(f"Attempt {attempt+1} returned empty response.")
            time.sleep(backoff * (attempt + 1))  # Exponential backoff

    return {'vehicleId': 'N/A', 'stationName': 'N/A'}  # Return default if all retries fail


In [2]:
import os
from dotenv import load_dotenv

load_dotenv() #load from your .env file
app_key = os.getenv('TFL_API_KEY')

stop="stockwell"
line_id = "victoria"
print(get_stop_point_info(stop))
stop_point_id = [id['id'] for id in get_stop_point_info(stop)] # gets the id out of the {name, id] result}
direction = "outbound"

sleep_time = 35

[{'name': 'Stockwell Underground Station', 'id': '940GZZLUSKW'}]


In [29]:
tube = get_tube_arrivals(line_id, stop_point_id[0], direction, app_key)
tube

[{'$type': 'Tfl.Api.Presentation.Entities.Prediction, Tfl.Api.Presentation.Entities',
  'id': '659974255',
  'operationType': 1,
  'vehicleId': '210',
  'naptanId': '940GZZLUSKW',
  'stationName': 'Stockwell Underground Station',
  'lineId': 'victoria',
  'lineName': 'Victoria',
  'platformName': 'Northbound - Platform 1',
  'direction': 'outbound',
  'bearing': '',
  'destinationNaptanId': '940GZZLUWWL',
  'destinationName': 'Walthamstow Central Underground Station',
  'timestamp': '2024-07-15T17:45:44.8619905Z',
  'timeToStation': 81,
  'currentLocation': 'Between Brixton and Stockwell',
  'towards': 'Walthamstow Central',
  'expectedArrival': '2024-07-15T17:47:05Z',
  'timeToLive': '2024-07-15T17:47:05Z',
  'modeName': 'tube',
  'timing': {'$type': 'Tfl.Api.Presentation.Entities.PredictionTiming, Tfl.Api.Presentation.Entities',
   'countdownServerAdjustment': '00:00:00',
   'source': '0001-01-01T00:00:00',
   'insert': '0001-01-01T00:00:00',
   'read': '2024-07-15T17:45:41.888Z',
  

In [ ]:
import logging
import time
from quixstreams import Application

def main(): 
    app = Application(
        broker_address="localhost:19092",
        loglevel="DEBUG",
    )

    with app.get_producer() as producer:
        while True:
            tube = get_tube_arrivals(line_id, stop_point_id[0], direction, app_key)
            # vehicleId, stationName, platformName, timestamp, timeToStation, currentLocation, towards
            if not tube:
                tube = ['N/A']
            vehicleId = tube['vehicleId'] if tube else 'N/A'
            logging.debug(f"Got vehicle: {vehicleId}")
            producer.produce(
                topic="tfl-tubes",
                key=tube['stationName'] if tube else 'N/A',
                value=json.dumps(tube),
                headers= {"app.name": "python-quix"}
            )
            logging.info(f"Produced. Sleeping for {sleep_time}s...")
            time.sleep(sleep_time)

#this for main, won't work in Notebook
if __name__ == "__main__":
    logging.basicConfig(level="DEBUG")
    main()